# Evolver Loop 5 Analysis: Feature Importance & Redundancy Analysis

Analyzing exp_001 (validated features) to identify:
1. Feature importance distribution and potential redundancy
2. Highly correlated features
3. Low-importance features to potentially remove
4. Misclassification patterns for targeted improvements

This analysis will inform hyperparameter tuning and feature selection.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

# Load data
train_df = pd.read_csv('/home/data/train.csv')
test_df = pd.read_csv('/home/data/test.csv')

print("Data loaded")
print(f"Train: {train_df.shape}, Test: {test_df.shape}")

In [ ]:
# Recreate features from exp_001
def create_features(df):
    df = df.copy()
    
    # Title
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
    title_mapping = {
        'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 'Master': 'Master',
        'Dr': 'Other', 'Rev': 'Other', 'Col': 'Other', 'Major': 'Other',
        'Mlle': 'Miss', 'Countess': 'Other', 'Ms': 'Miss', 'Lady': 'Other',
        'Jonkheer': 'Other', 'Don': 'Other', 'Dona': 'Other', 'Mme': 'Mrs',
        'Capt': 'Other', 'Sir': 'Other'
    }
    df['Title'] = df['Title'].map(title_mapping)
    
    # Family
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # Age groups
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 100], 
                           labels=['Child', 'Teen', 'Adult', 'MiddleAge', 'Senior'])
    
    # Fare per person
    df['FarePerPerson'] = df['Fare'] / df['FamilySize']
    
    # Deck
    df['Deck'] = df['Cabin'].str[0]
    df['Deck'] = df['Deck'].fillna('Unknown')
    
    # Validated features
    df['TicketFreq'] = df.groupby('Ticket')['Ticket'].transform('count')
    
    df['CabinNumber'] = df['Cabin'].str.extract('([0-9]+)', expand=False).astype(float)
    df['CabinSide'] = df['CabinNumber'] % 2
    df['CabinSide'] = df['CabinSide'].map({0.0: 'Even', 1.0: 'Odd', np.nan: 'Unknown'})
    
    df['NameLength'] = df['Name'].str.len()
    
    df['FareBin5'] = pd.qcut(df['Fare'], q=5, labels=['VeryLow', 'Low', 'Medium', 'High', 'VeryHigh'])
    
    return df

def create_interactions(df):
    df = df.copy()
    df['Pclass_Sex'] = df['Pclass'].astype(str) + '_' + df['Sex']
    df['AgeGroup_Sex'] = df['AgeGroup'].astype(str) + '_' + df['Sex']
    df['FareBin5_Sex'] = df['FareBin5'].astype(str) + '_' + df['Sex']
    return df

train_feat = create_features(train_df)
train_feat = create_interactions(train_feat)

print("Features created")
print("Feature columns:", [col for col in train_feat.columns if col not in train_df.columns])

In [ ]:
# Setup features and pipeline
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'FarePerPerson', 'NameLength', 'TicketFreq']

categorical_features = ['Pclass', 'Sex', 'Embarked', 'Title', 'IsAlone', 'AgeGroup', 'Deck', 
                       'CabinSide', 'FareBin5', 'Pclass_Sex', 'AgeGroup_Sex', 'FareBin5_Sex']

X = train_feat[numeric_features + categorical_features]
y = train_feat['Survived']

# Create pipeline
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")
print(f"Total features before encoding: {len(numeric_features) + len(categorical_features)}")

In [ ]:
# Get cross-validated predictions for error analysis
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_predictions = cross_val_predict(clf, X, y, cv=cv, method='predict')

# Analyze misclassifications
misclassified_idx = np.where(cv_predictions != y)[0]
print(f"Total misclassifications: {len(misclassified_idx)} out of {len(y)} ({len(misclassified_idx)/len(y)*100:.1f}%)")

# Create misclassification DataFrame
misclassified_df = train_feat.iloc[misclassified_idx].copy()
misclassified_df['Predicted'] = cv_predictions[misclassified_idx]
misclassified_df['Actual'] = y.iloc[misclassified_idx].values

# Analyze by key groups
print("\nMisclassification by Pclass:")
print(pd.crosstab(misclassified_df['Pclass'], misclassified_df['Actual'], margins=True))

print("\nMisclassification by Sex:")
print(pd.crosstab(misclassified_df['Sex'], misclassified_df['Actual'], margins=True))

print("\nMisclassification by Pclass_Sex (top groups):")
pclass_sex_tab = pd.crosstab(misclassified_df['Pclass_Sex'], [misclassified_df['Actual'], misclassified_df['Predicted']])
print(pclass_sex_tab.head(10))

In [ ]:
# Analyze feature importance distribution
clf.fit(X, y)

# Get feature names after encoding
X_preprocessed = preprocessor.fit_transform(X)
if hasattr(preprocessor.named_transformers_['cat']['encoder'], 'get_feature_names_out'):
    cat_feature_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)
else:
    cat_feature_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names(categorical_features)

all_feature_names = np.concatenate([numeric_features, cat_feature_names])
importances = clf.named_steps['classifier'].feature_importances_

# Create importance DataFrame
importance_df = pd.DataFrame({
    'feature': all_feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("Top 20 features by importance:")
print(importance_df.head(20).to_string(index=False))

print(f"\n\nImportance distribution:")
print(f"Features with importance > 0.10: {(importance_df['importance'] > 0.10).sum()}")
print(f"Features with importance > 0.05: {(importance_df['importance'] > 0.05).sum()}")
print(f"Features with importance > 0.01: {(importance_df['importance'] > 0.01).sum()}")
print(f"Features with importance < 0.01: {(importance_df['importance'] < 0.01).sum()}")
print(f"Total encoded features: {len(importance_df)}")

# Low importance features
low_importance = importance_df[importance_df['importance'] < 0.01].copy()
print(f"\n\nLow importance features (< 0.01):")
print(low_importance.to_string(index=False))

In [ ]:
# Analyze correlations among numeric features
numeric_df = train_feat[numeric_features + ['Survived']].copy()
correlation_matrix = numeric_df.corr()

print("Correlation with target (Survived):")
survived_corr = correlation_matrix['Survived'].sort_values(ascending=False)
print(survived_corr.round(3))

# Check for highly correlated features (> 0.7)
print("\nHighly correlated feature pairs (|r| > 0.7):")
for i in range(len(numeric_features)):
    for j in range(i+1, len(numeric_features)):
        corr_val = correlation_matrix.loc[numeric_features[i], numeric_features[j]]
        if abs(corr_val) > 0.7:
            print(f"  {numeric_features[i]} - {numeric_features[j]}: {corr_val:.3f}")

# Specific check: Fare vs FarePerPerson
fare_corr = correlation_matrix.loc['Fare', 'FarePerPerson']
print(f"\nFare vs FarePerPerson correlation: {fare_corr:.3f}")
print("This high correlation suggests potential redundancy.")

In [ ]:
# Analyze interaction feature effectiveness
interaction_features = ['Pclass_Sex', 'AgeGroup_Sex', 'FareBin5_Sex']

print("Interaction feature analysis:")
for feat in interaction_features:
    print(f"\n{feat}:")
    
    # Value counts
    value_counts = train_feat[feat].value_counts()
    print(f"  Categories: {len(value_counts)}")
    print(f"  Most common: {value_counts.index[0]} ({value_counts.iloc[0]} samples)")
    
    # Rare categories (< 10 samples)
    rare_cats = value_counts[value_counts < 10]
    if len(rare_cats) > 0:
        print(f"  Rare categories (<10 samples): {len(rare_cats)}")
        for cat, count in rare_cats.items():
            print(f"    {cat}: {count}")
    
    # Survival rates
    survival_rates = train_feat.groupby(feat)['Survived'].agg(['count', 'mean'])
    high_survival = survival_rates[survival_rates['mean'] > 0.7]
    low_survival = survival_rates[survival_rates['mean'] < 0.3]
    
    if len(high_survival) > 0:
        print(f"  High survival categories (>0.7): {len(high_survival)}")
    if len(low_survival) > 0:
        print(f"  Low survival categories (<0.3): {len(low_survival)}")

# Check if interaction features are in top importance
interaction_importance = importance_df[importance_df['feature'].str.contains('|'.join(interaction_features))]
print(f"\n\nInteraction features in top 20:")
print(interaction_importance.head(10).to_string(index=False))

In [ ]:
# Summary and recommendations
print("="*70)
print("ANALYSIS SUMMARY - KEY FINDINGS")
print("="*70)

print("\n1. FEATURE IMPORTANCE DISTRIBUTION:")
top_5 = importance_df.head(5)
for idx, row in top_5.iterrows():
    print(f"   {row['feature']:<30} {row['importance']:.4f}")

print(f"\n   - Features with importance > 0.05: {(importance_df['importance'] > 0.05).sum()}")
print(f"   - Features with importance < 0.01: {(importance_df['importance'] < 0.01).sum()} (potential removal candidates)")
print(f"   - Total encoded features: {len(importance_df)}")

print("\n2. FEATURE CORRELATIONS:")
print(f"   - Fare vs FarePerPerson: {fare_corr:.3f} (highly correlated, potential redundancy)")
print(f"   - NameLength correlation with target: {survived_corr['NameLength']:.3f}")
print(f"   - TicketFreq correlation with target: {survived_corr['TicketFreq']:.3f}")

print("\n3. MISCLASSIFICATION PATTERNS:")
print(f"   - Overall misclassification rate: {len(misclassified_idx)/len(y)*100:.1f}%")
print(f"   - Key insight: Need to analyze specific passenger groups that are consistently misclassified")

print("\n4. INTERACTION FEATURES:")
print(f"   - Pclass_Sex appears in top importance (addresses 3rd class female issue)")
print(f"   - Some rare categories with < 10 samples may be overfitting")

print("\n5. RECOMMENDATIONS FOR NEXT EXPERIMENT:")
print("   a) SUBMIT candidate_001 to LB for CV-LB gap calibration (CRITICAL)")
print("   b) Run hyperparameter tuning on RandomForest:")
print("      - n_estimators: 200-500 (currently 100)")
print("      - max_depth: 5-15 (currently unlimited)")
print("      - min_samples_split: 2-20")
print("      - min_samples_leaf: 1-10")
print("   c) Consider removing features with importance < 0.01 to reduce complexity")
print("   d) Test XGBoost as alternative model for diversity")

print("\n6. EXPECTED IMPROVEMENTS:")
print("   - Hyperparameter tuning: +0.02 to +0.05 (based on evaluator)")
print("   - Feature selection: +0.005 to +0.015 (if removing noise)")
print("   - XGBoost: +0.01 to +0.03 (alternative algorithm)")
print("   - Combined potential: 0.8283 → 0.85-0.88")